In [1]:
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import requests
from requests import get
import re

# Pulling all links off of the search pages (up to 3000) and putting them in a dataframe to scrape.

In [2]:
url_template="http://www.indeed.com/jobs?q=%22Data+Analyst%22&l=Seattle%2C+WA&start={}"
max_results=250
Linkdf=[]

for start in range(0, max_results, 7):
    url=url_template.format(start)
    html=requests.get(url)
    soup=BeautifulSoup(html.content,'html.parser', from_encoding="utf-8")
    
    #for each in soup.find_all(a_="href"):
    page_links=soup.find_all('a',{'href':re.compile("/rc/")})
    for items in page_links:
        Linkdf.append(items['href'])   

In [3]:
#Sanity Check
len(Linkdf)
#print(Linkdf)

334

In [4]:
#This code allows the code to display the full website instead of truncating
pd.set_option('display.max_colwidth',-1)

#Moving it to a data frame
data = {'links':Linkdf}
df = pd.DataFrame(data, columns=['links'])

#append indeed.com to the front of each
df['Web'] = 'https://www.indeed.com'
df['URL'] = df.Web.str.cat(df.links)

#pull out just a list of the websites.
websites=list(df['URL'])

#Sanity Check
#print(websites)
len(websites)

334

In [5]:
websites1=set(websites)

In [6]:
len(websites1)

169

# Looping through websites....

In [7]:
Title=[]
Descriptions=[]
Location=[]
FullDescriptions=[]

for url in websites1:
    response=get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    description_containers= soup.find(class_='jobsearch-jobDescriptionText')
    title_containers=soup.find('h3')
    try:
        location_containers=soup.find('',{'class':'jobsearch-CompanyInfoWithoutHeaderImage'}).find_all('div')[-1]
    except:
        location_containers='None Found'
    
    job_descriptions=str(description_containers)
    job_title=str(title_containers.text)
    try:
        locations=str(location_containers.text)
    except AttributeError:
        locations = 'None Found'
    try:
        full_descriptions = str(description_containers.text)
    except AttributeError:
        full_descriptions= 'None Found'
    
    Descriptions.append(job_descriptions)
    Title.append(job_title)
    Location.append(locations)
    FullDescriptions.append(full_descriptions)    

# Extracting what we want from the Descriptions Column

In [8]:
#'Location' left in for sanity check. Should be removed once code is confirmed to work
Descriptions_df = pd.DataFrame(columns = ['Title', 'Location','City', 'State', 'Zip', 'Country', 'Qualifications', 'Skills', 'Responsibilities', 'Education', 'Requirement', 'FullDescriptions'])
Country = ['US', 'USA', 'United States', 'United States of Americal']
States = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA',
          'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',
          'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

for index, element in enumerate(Descriptions):
    soup=BeautifulSoup(element,'lxml')
    for values in list(Descriptions_df):
        temp_tag = soup.find('b', text=re.compile(values))
        try:
            ul_tag = temp_tag.find_next('ul')
            Descriptions_df.at[index,values] = ul_tag.text
        except AttributeError:
            Descriptions_df.at[index,values]="None Found"
        Descriptions_df.at[index,"Title"]=Title[index]
        Descriptions_df.at[index,"Location"]=Location[index]
        Descriptions_df.at[index,"FullDescriptions"]=FullDescriptions[index]
        words = '|'.join(Country)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Country"] = temp[0]
        words = '|'.join(States)
        temp = re.findall(words, Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"State"] = temp[0]
        temp = re.findall(r'\d+', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"Zip"] = temp[0]  
            
        temp = re.findall(r'[\w w]+,', Location[index])
        if len(temp) != 0:
            Descriptions_df.at[index,"City"] = re.sub(',', '', temp[0])


In [9]:
Descriptions_df

Title           Location      City State  \
0    Business Intelligence Analyst      Bellevue, WA       Bellevue  WA     
1    Senior Technical Business Analyst  Redmond, WA        Redmond   WA     
2    Product Manager                    Seattle, WA        Seattle   WA     
3    Master Data Analyst III            Kent, WA           Kent      WA     
4    SOFTWARE ENGINEER                  Seattle, WA        Seattle   WA     
..                 ...                          ...            ...   ..     
164  Cost Analyst/Accountant            Everett, WA 98204  Everett   WA     
165  DIRECTOR OF RETENTION              Seattle, WA 98104  Seattle   WA     
166  RESEARCH CONSULTANT                Seattle, WA        Seattle   WA     
167  Sales Data Analyst                 Redmond, WA        Redmond   WA     
168  CORE - Data Analyst                Seattle, WA 98115  Seattle   WA     

            Zip     Country  \
0    None Found  None Found   
1    None Found  None Found   
2    None Found  None Found   
3    None Found  None Found   
4    None Found  None Found   
..          ...         ...   
164  98204       None Found   
165  98104       None Found   
166  None Found  None Found   
167  None Found  None Found   
168  98115       None Found   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 Qualifications  \
0    None Found                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
1    \n5 or more years of experience in a data analyst role.\nExperience with agile principles & practices, GIT/VSO preferred.\nAbility to understand and translate business and product questions into data & analytics projects.\nAbility to articulate vision, requirements, and benefits to business and engineering partners.\nEffective communication skills and the ability to work collaboratively with stakeholders, partner engineering teams, and subject matter experts.\nCreativity to find pragmatic paths through ambiguity.\nAbility to communicate and collaborate with people representing diverse points of view.\nWillingness to work in a start-up type environment with evolving responsibilities and a wide variety of work.\nBachelors degree in Computer Science, Engineering, Mathematics or related fields, or sufficient experience in these areas.   
2    None Found                                                                                                                                                                       

# Move to csv

In [10]:
Descriptions_df.to_csv('Descriptions_df_DataAnalyst_Seattle.csv')